In [ ]:
#gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Libraries

In [ ]:
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 9.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install torch torchvision transformers

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=8b526c1291952dd5c3a3ea5129e94831c7ecc1797bd8b87832d2aa0479ea7325
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:

#imports
import os
import json
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, BertTokenizer, BertModel, T5Config
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
from sklearn.model_selection import train_test_split
import tensorflow as tf
import torch
import re

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from transformers import (BertTokenizer, BertModel, T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq, get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup)
import torch.nn.functional as F
import torch.nn as nn
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
# from sklearn.metrics import accuracy_score, f1_score
# import numpy as np
# from datasets import load_metric


In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.cuda.is_available()

True

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(f"Using device: {device}")

Using device: cuda


## Simple

In [ ]:
#lists
json_data=[]
prompts=[]

In [ ]:
# load json-only
json_only= '/content/drive/My Drive/Implementation/Final Thesis/json_only.jsonl'

In [ ]:
with open(json_only, 'r', encoding='utf-8') as f:
  for line in f:
    json_data.append(json.loads(line))

In [ ]:
# load prompt only
prompt_only= '/content/drive/My Drive/Implementation/Final Thesis/prompt_only.txt'

In [ ]:
with open(prompt_only, 'r', encoding='utf-8') as f:
  prompts =[line.strip() for line in f]

In [ ]:
print(f"Number of prompts: {len(prompts)}")
print(f"Number of JSON entries: {len(json_data)}")

Number of prompts: 28085
Number of JSON entries: 28085


In [ ]:
# Example of a JSON entry
print("Example JSON entry:")
print(json.dumps(json_data[0], indent=2))

Example JSON entry:
{
  "variant_properties": {
    "color": "rgba(255, 255, 255, 1.0)",
    "strokes": [
      "rgba(126, 86, 216, 1.0)"
    ],
    "strokeWeight": 1.0,
    "text": "Button CTA",
    "textColor": "rgba(255, 255, 255, 1.0)",
    "borderRadius": 10.0,
    "fontFamily": "Inter",
    "fontWeight": 500,
    "fontSize": 14.0,
    "effects": [
      {
        "type": "DROP_SHADOW",
        "color": "rgba(16, 24, 40, 0.05000000074505806)"
      }
    ],
    "padding": 0,
    "width": 77.0,
    "height": 20.0,
    "x": -4619.0,
    "y": -2135.0,
    "style": "Professional",
    "component_name": "Button",
    "subtype": "Default",
    "variant_details": {
      "State": [
        "Default"
      ],
      "Size": [
        "Small"
      ]
    }
  }
}


In [ ]:
class UIDataset(Dataset):
   """ class for loading paired dataset """

  def __init__(self, prompts, json_data):
    """method to initialize the class
    Parameters:
    prompts (list): list of prompts
    json_data (list): list of JSON data
    """
    #initialise prompts
    self.prompts= prompts
    #initialise json data
    self.json_data= json_data
    #loading bert tokenizer
    self.bert_tokenizer= BertTokenizer.from_pretrained('bert-base-uncased')
    self.t5_tokenizer= T5Tokenizer.from_pretrained('t5-small')
    #bert model
    self.bert_model= BertModel.from_pretrained('bert-base-uncased').to(device)

  def __len__(self):
    """ method to return the length of the dataset """
    return len(self.prompts)

  def __getitem__(self, idx):
    """ method to return the item at the given index
    Parameters:
    idx (int): index of the item
    Returns:
    dict: dictionary containing the input_ids, attention_mask, labels, and bert_embeddings
    """
    #indices of promtp and json
    prompt= self.prompts[idx]
    json_entry =json.dumps(self.json_data[idx])

    #bert used to convert prompts to tokens
    inputs= self.bert_tokenizer(prompt, return_tensors='pt', padding='max_length', truncation=True, max_length=512).to(device)
    #bert embeddings generated without gradient
    with torch.no_grad():
        outputs =self.bert_model(**inputs)
    #average last hidden states from bert to get a single embedding vector for the prompt
    bert_embeddings= outputs.last_hidden_state.mean(dim=1).squeeze()

    # check for NaN in embeddings
    if torch.isnan(bert_embeddings).any():
      print(f"NaN detected in BERT embeddings at index {idx}")

    #t5 tokenizer to conver prompt to tokens
    t5_inputs= self.t5_tokenizer(prompt, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
    #t5 tokenizer to convert json to tokens
    targets =self.t5_tokenizer(json_entry, return_tensors='pt', padding='max_length', truncation=True, max_length=512)

    return {
      'input_ids':t5_inputs['input_ids'].squeeze(),
      'attention_mask':t5_inputs['attention_mask'].squeeze(),
      'labels': targets['input_ids'].squeeze(),
      'bert_embeddings':bert_embeddings
    }

In [ ]:
dataset= UIDataset(prompts, json_data)
#split the dataset
total_size= len(dataset)
train_size= int(0.8 * total_size)
val_size= int(0.1 * total_size)
test_size= total_size - train_size - val_size

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
train_dataset, val_dataset, test_dataset= random_split(dataset, [train_size, val_size, test_size])

In [ ]:
#dataloaders
train_loader= DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader= DataLoader(val_dataset, batch_size=4)
test_loader= DataLoader(test_dataset, batch_size=4)


In [ ]:
#tokenizers and models bert
bert_tokenizer =BertTokenizer.from_pretrained('bert-base-uncased')
bert_model= BertModel.from_pretrained('bert-base-uncased')


In [ ]:
#tokenizers and models t5
t5_tokenizer= T5Tokenizer.from_pretrained('t5-small')
t5_model =T5ForConditionalGeneration.from_pretrained('t5-small')

#move to device
device= 'cuda' if torch.cuda.is_available() else 'cpu'
bert_model.to(device)
t5_model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
# Training arguments
training_args= TrainingArguments(
  output_dir='/content/drive/My Drive/Implementation/Final Thesis/FYP/JSON generation/T5 CROSS SIMPLE',
  num_train_epochs=3,
  #training batch size
  per_device_train_batch_size=4,
  #evaluation batch size
  per_device_eval_batch_size=4,
  save_steps=500,
  save_total_limit=2,
  logging_steps=10,
  eval_steps=500,
  evaluation_strategy="steps",
  prediction_loss_only=True,
  load_best_model_at_end=True,
  fp16=torch.cuda.is_available(),
  #not push to HF
  push_to_hub=False
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
#DATALOADER for seq2seq
data_collator= DataCollatorForSeq2Seq(tokenizer=t5_tokenizer, model=t5_model)

In [ ]:
class T5WithCrossAttention(T5ForConditionalGeneration):
  """ class for T5 model with cross attention """
  def __init__(self, config):
    """ method to initialize the class
    Parameters:
    config (dict): configuration of the model
    """
    super().__init__(config)
    #initialise multi head attention layer for cross attention
    self.cross_attention =torch.nn.MultiheadAttention(embed_dim=config.d_model, num_heads=8)
    #initialise linear layer to put out bert embeddings to t5 hidden state dimension
    self.bert_proj= torch.nn.Linear(bert_model.config.hidden_size, config.d_model)

  def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, labels=None, bert_embeddings=None):
    """ method to forward the input
    Parameters:
    input_ids (tensor): input tokens
    attention_mask (tensor): attention mask
    decoder_input_ids (tensor): decoder input tokens
    labels (tensor): labels
    bert_embeddings (tensor): bert embeddings
    Returns:
    tuple: loss and logits
    """
    #pass inputs through t5 encoder
    outputs= self.encoder(input_ids=input_ids,attention_mask=attention_mask,)

    hidden_states= outputs[0]

    #bert embeddings to match the T5 hidden state dimension
    bert_embeddings= self.bert_proj(bert_embeddings).unsqueeze(1).expand(-1, hidden_states.size(1), -1)

    #putting cross-attention between T5 embeddings and BERT embeddings
    hidden_states, _= self.cross_attention(hidden_states, bert_embeddings, bert_embeddings)

    #pass inputs through t5 decoder
    decoder_outputs= self.decoder(input_ids=decoder_input_ids,encoder_hidden_states=hidden_states,encoder_attention_mask=attention_mask,)

    sequence_output = decoder_outputs[0]
    lm_logits = self.lm_head(sequence_output)

    loss = None
    if labels is not None:
      loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
      #flatten logits and labels for loss computation
      lm_logits = lm_logits.view(-1, lm_logits.size(-1))
      #labels
      labels = labels.view(-1)
      if lm_logits.size(0)== labels.size(0):
        loss = loss_fct(lm_logits, labels)
      else:
        print(f"Shape mismatch between logits and labels: {lm_logits.size()} vs {labels.size()}")

    return loss, lm_logits


In [ ]:
# t5_model = T5WithCrossAttention.from_pretrained('t5-small').to(device)

In [ ]:
#Trainer instance
trainer = Trainer(
  model=t5_model,
  args=training_args,
  train_dataset=train_dataset,
  eval_dataset=val_dataset,
  data_collator=data_collator
)

In [ ]:
#path
checkpoint_path= "/content/drive/My Drive/Implementation/Final Thesis/FYP/UpdatedResults/checkpoint-16500"

In [ ]:
#LOAD
loaded_model= T5ForConditionalGeneration.from_pretrained(checkpoint_path).to(device)

In [ ]:
original_model_path= "t5-small"

In [ ]:
#LOAD TOKENIZER
loaded_tokenizer= T5Tokenizer.from_pretrained(original_model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Inference

In [ ]:
def generate_json_from_description(description, model, tokenizer, max_length=512):
  """ method to generate JSON from description
  Parameters:
  description (str): description
  model (T5ForConditionalGeneration): model
  tokenizer (T5Tokenizer): tokenizer
  max_length (int): maximum length of the generated JSON
  Returns:
  str: generated JSON
  """
  #tokenize input description
  input_ids= tokenizer(description, return_tensors="pt", padding="max_length", truncation=True, max_length=max_length).input_ids.to(device)
  #generate JSON output
  generated_ids= model.generate(input_ids, max_length=max_length, num_beams=5, early_stopping=True)
  #decode generated tokens to JSON string
  generated_json =tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return generated_json

In [ ]:
description="Create a professional label with a drop shadow effect"

In [ ]:
generated_json= generate_json_from_description(description, loaded_model, loaded_tokenizer)


In [ ]:
print(generated_json)

"variant_properties": "color": "rgba(255, 255, 255, 1.0)", "strokes": [], "strokeWeight": 1.0, "text": "Label", "textColor": "rgba(255, 255, 255, 1.0)", "borderRadius": 8.0, "fontFamily": "Roboto", "fontWeight": 500, "fontSize": 14.0, "effects": ["type": "DROP_SHADOW", "color": "rgba(0, 0, 0, 0.30000001192092896)", "type": "DROP_SHADOW", "color": "rgba(0, 0, 0, 0.15000000596046448)"], "padding": 0, "width": 35.0, "height": 20.0, "x": -5349.0, "y": -4313.0, "style": "Professional", "component_name": "label", "subtype": "Light", "variant_details": "Style": ["outlined"], "State": ["dragged"]


In [ ]:
# save_path= "/content/drive/My Drive/Implementation/Final Thesis/FYP/UpdatedResults/unfinished_model"

In [ ]:
# if not os.path.exists(save_path):
#   os.makedirs(save_path)

In [ ]:
# loaded_model.save_pretrained(save_path)
# loaded_tokenizer.save_pretrained(save_path)

('/content/drive/My Drive/Implementation/Final Thesis/FYP/UpdatedResults/unfinished_model/tokenizer_config.json',
 '/content/drive/My Drive/Implementation/Final Thesis/FYP/UpdatedResults/unfinished_model/special_tokens_map.json',
 '/content/drive/My Drive/Implementation/Final Thesis/FYP/UpdatedResults/unfinished_model/spiece.model',
 '/content/drive/My Drive/Implementation/Final Thesis/FYP/UpdatedResults/unfinished_model/added_tokens.json')

## Training again1 with pairs

In [ ]:
# Load the paired dataset
pairs_file_1= '/content/drive/My Drive/Implementation/Final Thesis/FYP/Data/t5_json_description_pairs.jsonl'


In [ ]:

data_pairs_1= []


In [ ]:
with open(pairs_file_1, 'r', encoding='utf-8') as f:
  for line in f:
    data_pairs_1.append(json.loads(line))


In [ ]:
json_data_1 =[pair['input'] for pair in data_pairs_1]


In [ ]:
prompts_1= [pair['output'] for pair in data_pairs_1]


In [ ]:
print(f"Number of pairs: {len(data_pairs_1)}")


Number of pairs: 28300


In [ ]:
print(f"Number of jsno: {len(json_data_1)}")


Number of jsno: 28300


In [ ]:
print(f"Number of description: {len(prompts_1)}")


Number of description: 28300


In [ ]:
for i in range(2):
  print(f"JSON: {json_data_1[i]}")
  print(f"Description: {prompts_1[i]}")


JSON: {"variant_properties": {"color": "rgba(255, 255, 255, 1.0)", "strokes": ["rgba(126, 86, 216, 1.0)"], "strokeWeight": 1.0, "text": "Button CTA", "textColor": "rgba(255, 255, 255, 1.0)", "borderRadius": 10.0, "fontFamily": "Inter", "fontWeight": 500, "fontSize": 14.0, "effects": [{"type": "DROP_SHADOW", "color": "rgba(16, 24, 40, 0.05000000074505806)"}], "padding": 0, "width": 77.0, "height": 20.0, "x": -4619.0, "y": -2135.0, "hasIcon": false, "style": "Professional", "component_name": "Button", "subtype": "Default", "variant_details": {"State": ["Default"], "Size": ["Small"]}}}
Description: A Button with border radius 10.0.
JSON: {"variant_properties": {"color": "rgba(255, 255, 255, 1.0)", "strokes": ["rgba(126, 86, 216, 1.0)"], "strokeWeight": 1.0, "text": "Button CTA", "textColor": "rgba(255, 255, 255, 1.0)", "borderRadius": 10.0, "fontFamily": "Inter", "fontWeight": 500, "fontSize": 14.0, "effects": [{"type": "DROP_SHADOW", "color": "rgba(16, 24, 40, 0.05000000074505806)"}], "p

In [ ]:
class UIDataset(Dataset):
  """ class for loading paired dataset """

  def __init__(self, prompts, json_data):
    """method to initialize the class
    Parameters:
    prompts (list): list of prompts
    json_data (list): list of JSON data
    """
    #initialise prompts
    self.prompts= prompts
    #initialise json data
    self.json_data=json_data
    #loading bert tokenizer
    self.bert_tokenizer= BertTokenizer.from_pretrained('bert-base-uncased')
    self.t5_tokenizer =T5Tokenizer.from_pretrained('t5-small')
    #bert model
    self.bert_model= BertModel.from_pretrained('bert-base-uncased').to(device)

  def __len__(self):
    """ method to return the length of the dataset """
    return len(self.prompts)

  def __getitem__(self, idx):
    """ method to return the item at the given index
    Parameters:
    idx (int): index of the item
    Returns:
    dict: dictionary containing the input_ids, attention_mask, labels, and bert_embeddings
    """
    #indices of promtp and json
    prompt= self.prompts[idx]
    json_entry =json.dumps(self.json_data[idx])

    #bert used to convert prompts to tokens
    inputs= self.bert_tokenizer(prompt, return_tensors='pt', padding='max_length', truncation=True, max_length=512).to(device)
    #bert embeddings generated without gradient
    with torch.no_grad():
      outputs = self.bert_model(**inputs)
    #average last hidden states from bert to get a single embedding vector for the prompt
    bert_embeddings= outputs.last_hidden_state.mean(dim=1).squeeze()

    # check for NaN in embeddings
    if torch.isnan(bert_embeddings).any():
      print(f"NaN detected in BERT embeddings at index {idx}")

    #t5 tokenizer to conver prompt to tokens
    t5_inputs= self.t5_tokenizer(prompt, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
    #t5 tokenizer to convert json to tokens
    targets = self.t5_tokenizer(json_entry, return_tensors='pt', padding='max_length', truncation=True, max_length=512)

    return {
      'input_ids':t5_inputs['input_ids'].squeeze(),
      'attention_mask':t5_inputs['attention_mask'].squeeze(),
      'labels': targets['input_ids'].squeeze(),
      'bert_embeddings': bert_embeddings
    }

In [ ]:
# Split the dataset
dataset= UIDataset(prompts_1, json_data_1)
total_size =len(dataset)
train_size= int(0.8 * total_size)
val_size =int(0.1 * total_size)
test_size= total_size - train_size - val_size

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

In [ ]:
train_dataset, val_dataset, test_dataset= random_split(dataset, [train_size, val_size, test_size])

In [ ]:
#dataloaders
train_loader= DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader= DataLoader(val_dataset, batch_size=4)
test_loader= DataLoader(test_dataset, batch_size=4)


In [ ]:
#tokenizers and models bert
bert_tokenizer =BertTokenizer.from_pretrained('bert-base-uncased')
bert_model =BertModel.from_pretrained('bert-base-uncased')


In [ ]:
#tokenizers and models t5
t5_tokenizer= T5Tokenizer.from_pretrained('t5-small')
t5_model= T5ForConditionalGeneration.from_pretrained('t5-small')

#move to device
device= 'cuda' if torch.cuda.is_available() else 'cpu'
bert_model.to(device)
t5_model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
epochs=10

In [ ]:
# Training arguments
training_args= TrainingArguments(
  output_dir='/content/drive/My Drive/Implementation/Final Thesis/FYP/JSON generation/T5 CROSS SIMPLE',
  num_train_epochs=epochs,
  #training batch size
  per_device_train_batch_size=16,
  #evaluation batch size
  per_device_eval_batch_size=16,
  save_steps=500,
  save_total_limit=2,
  logging_steps=10,
  eval_steps=500,
  evaluation_strategy="steps",
  #loss calculated during validation
  prediction_loss_only=True,
  load_best_model_at_end=True,
  fp16=torch.cuda.is_available(),
  #not push to HF
  push_to_hub=False
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
data_collator= DataCollatorForSeq2Seq(tokenizer=t5_tokenizer, model=t5_model)

In [ ]:
class T5WithCrossAttention(T5ForConditionalGeneration):
  """ class for T5 model with cross attention """
  def __init__(self, config):
    """ method to initialize the class
    Parameters:
    config (dict): configuration of the model
    """
    super().__init__(config)
    #initialise multi head attention layer for cross attention
    self.cross_attention= torch.nn.MultiheadAttention(embed_dim=config.d_model, num_heads=8)
    #initialise linear layer to put out bert embeddings to t5 hidden state dimension
    self.bert_proj = torch.nn.Linear(bert_model.config.hidden_size, config.d_model)

  def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, labels=None, bert_embeddings=None):
    """ method to forward the input
    Parameters:
    input_ids (tensor): input tokens
    attention_mask (tensor): attention mask
    decoder_input_ids (tensor): decoder input tokens
    labels (tensor): labels
    bert_embeddings (tensor): bert embeddings
    Returns:
    tuple: loss and logits
    """
    #pass inputs through t5 encoder
    outputs= self.encoder(input_ids=input_ids,attention_mask=attention_mask,)
    hidden_states= outputs[0]

    #bert embeddings to match the T5 hidden state dimension
    bert_embeddings= self.bert_proj(bert_embeddings).unsqueeze(1).expand(-1, hidden_states.size(1), -1)

    #putting cross-attention between T5 embeddings and BERT embeddings
    hidden_states, _= self.cross_attention(hidden_states, bert_embeddings, bert_embeddings)

    #pass inputs through t5 decoder
    decoder_outputs= self.decoder(input_ids=decoder_input_ids,encoder_hidden_states=hidden_states,encoder_attention_mask=attention_mask,)

    sequence_output= decoder_outputs[0]
    lm_logits= self.lm_head(sequence_output)

    loss = None
    if labels is not None:
      loss_fct= torch.nn.CrossEntropyLoss(ignore_index=-100)
      #flatten logits and labels for loss computation
      lm_logits= lm_logits.view(-1, lm_logits.size(-1))
      #labels
      labels =labels.view(-1)
      if lm_logits.size(0)== labels.size(0):
        loss = loss_fct(lm_logits, labels)
      else:
        print(f"Shape mismatch between logits and labels: {lm_logits.size()} vs {labels.size()}")

    return loss, lm_logits


In [ ]:
#Trainer instance
trainer = Trainer(
  model=t5_model,
  args=training_args,
  train_dataset=train_dataset,
  eval_dataset=val_dataset,
  data_collator=data_collator
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
500,0.458900,0.356550
1000,0.353000,0.264545
1500,0.292400,0.226589


Step,Training Loss,Validation Loss
500,0.458900,0.356550
1000,0.353000,0.264545
1500,0.292400,0.226589
2000,0.248600,0.202615
2500,0.223800,0.186282
3000,0.203500,0.174163
3500,0.209700,0.164607
4000,0.185800,0.157668
4500,0.183800,0.152535
5000,0.199200,0.148562


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=7075, training_loss=0.2927687621537872, metrics={'train_runtime': 6545.7361, 'train_samples_per_second': 17.294, 'train_steps_per_second': 1.081, 'total_flos': 1.53206919266304e+16, 'train_loss': 0.2927687621537872, 'epoch': 5.0})

In [ ]:
#path
checkpoint_path= "/content/drive/My Drive/Implementation/Final Thesis/FYP/JSON generation/T5 CROSS SIMPLE.t5_cross_model.pth"

In [ ]:
#save for future
torch.save(t5_model.state_dict(), checkpoint_path)

In [ ]:
#LOAD
t5_model= T5ForConditionalGeneration.from_pretrained('t5-small')
t5_tokenizer= T5Tokenizer.from_pretrained('t5-small')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
t5_model.load_state_dict(torch.load(checkpoint_path))

<All keys matched successfully>

In [ ]:
t5_model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

### Retrain

In [ ]:
#train again
trainer = Trainer(
  model=t5_model,
  args=training_args,
  train_dataset=train_dataset,
  eval_dataset=val_dataset,
  data_collator=data_collator
)

In [ ]:
#resume from checkpoint
trainer.train(resume_from_checkpoint=True)

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss
7500,0.179500,0.135771
8000,0.172800,0.132057
8500,0.153800,0.128165
9000,0.160000,0.125457
9500,0.150700,0.122636
10000,0.144800,0.120265
10500,0.149100,0.118214
11000,0.152300,0.116671
11500,0.147500,0.115402
12000,0.163700,0.114268


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=14150, training_loss=0.0781043372390127, metrics={'train_runtime': 6583.0359, 'train_samples_per_second': 34.391, 'train_steps_per_second': 2.149, 'total_flos': 3.06413838532608e+16, 'train_loss': 0.0781043372390127, 'epoch': 10.0})

In [ ]:
def generate_json_from_description(description, model, tokenizer, max_length=512):
  """method to generate JSON from description
  Parameters:
  description (str): description
  model (T5ForConditionalGeneration): model
  tokenizer (T5Tokenizer): tokenizer
  max_length (int): maximum length of the generated JSON
  Returns:
  str: generated JSON"""
  #tokenize input description
  input_ids= tokenizer(description, return_tensors="pt", padding="max_length", truncation=True, max_length=max_length).input_ids.to(device)
  #generate JSON output
  generated_ids= model.generate(input_ids, max_length=max_length, num_beams=5, early_stopping=True)
  #decode generated tokens to JSON string
  generated_json= tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return generated_json

In [ ]:
description="Create a professional button with a state of hover"

In [ ]:
generated_json= generate_json_from_description(description, t5_model, t5_tokenizer)


In [ ]:
print(generated_json)

""variant_properties": "color": "rgba(255, 255, 255, 1.0)", "strokes": ["rgba(255, 255, 255, 1.0)"], "strokeWeight": 1.0, "text": "Button", "textColor": "rgba(255, 255, 255, 1.0)", "borderRadius": 9999.0, "fontFamily": "SF Pro Text", "fontWeight": 400, "fontSize": 14.0, "effects": ["type": "DROP_SHADOW", "color": "rgba(5, 145, 255, 0.10000000149011612)"], "padding": 0, "width": 44.0, "height": 24.0, "x": -4749.0, "y": -4749.0, "hasIcon": true, "style": "Professional", "component_name": "Button", "subtype": "Default", "variant_details": "Size": ["Large"], "State": ["hover"]"


## Nested

In [ ]:
#lists
json_data=[]
prompts=[]

In [ ]:
# load json-only
json_only_input= '/content/drive/My Drive/Implementation/Final Thesis/FYP/Data/nested_dataset.jsonl'

In [ ]:
# json_only= '/content/drive/My Drive/Implementation/Final Thesis/FYP/Data/nested_dataset_5.jsonl'

In [ ]:
jsons = []

In [ ]:
# with open(json_only_input, 'r', encoding='utf-8') as f:
#   for line in f:
#     json_data = json.loads(line)

#     for _ in range(5):
#         jsons.append(json_data)

In [ ]:
with open(json_only, 'w', encoding='utf-8') as f:
  for data in jsons:
    f.write(json.dumps(data) + '\n')

In [ ]:
# load prompt only
prompt_only='/content/drive/My Drive/Implementation/Final Thesis//FYP/Data/t5_descriptions.txt'

In [ ]:
with open(prompt_only, 'r', encoding='utf-8') as f:
  prompts=[line.strip() for line in f]

In [ ]:
print(f"Number of prompts: {len(prompts)}")
print(f"Number of JSON entries: {len(jsons)}")

Number of prompts: 28240
Number of JSON entries: 28240


In [ ]:
if len(prompts)!= len(jsons):
    raise ValueError("The number of prompts and JSON entries must be the same")

In [ ]:
class UIDataset(Dataset):
  """ class for loading paired dataset """
  def __init__(self, prompts, json_data):
    """ method to initialize the class
    Parameters:
    prompts (list): list of prompts
    json_data (list): list of JSON data
    """
    #initialise prompts
    self.prompts = prompts
    #initialise json data
    self.json_data = json_data
    #loading bert tokenizer
    self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    self.t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')
    #bert model
    self.bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

  def __len__(self):
    """ method to return the length of the dataset """
    return len(self.prompts)

  def __getitem__(self, idx):
    """ method to return the item at the given index
    Parameters:
    idx (int): index of the item
    Returns:
    dict: dictionary containing the input_ids, attention_mask, labels, and bert_embeddings
    """
    #indices of prompt and json
    prompt = self.prompts[idx]
    json_entry = json.dumps(self.json_data[idx])

    #bert used to convert prompts to tokens
    inputs = self.bert_tokenizer(prompt, return_tensors='pt', padding='max_length', truncation=True, max_length=512).to(device)
     #bert embeddings generated without gradient
    with torch.no_grad():
        outputs = self.bert_model(**inputs)
    #average last hidden states from bert to get a single embedding vector for the prompt
    bert_embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()

    # check for NaN in embeddings
    if torch.isnan(bert_embeddings).any():
      print(f"NaN detected in BERT embeddings at index {idx}")

     #t5 tokenizer to conver prompt to tokens
    t5_inputs = self.t5_tokenizer(prompt, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
    #t5 tokenizer to convert json to tokens
    targets = self.t5_tokenizer(json_entry, return_tensors='pt', padding='max_length', truncation=True, max_length=512)

    return {
      'input_ids': t5_inputs['input_ids'].squeeze(),
      'attention_mask': t5_inputs['attention_mask'].squeeze(),
      'labels': targets['input_ids'].squeeze(),
      'bert_embeddings': bert_embeddings
    }

In [ ]:
# Split the dataset
dataset= UIDataset(prompts, jsons)
total_size= len(dataset)
train_size =int(0.8 * total_size)
val_size= int(0.1 * total_size)
test_size = total_size - train_size - val_size

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
train_dataset, val_dataset, test_dataset= random_split(dataset, [train_size, val_size, test_size])

In [ ]:
#dataloaders
train_loader= DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader= DataLoader(val_dataset, batch_size=4)
test_loader= DataLoader(test_dataset, batch_size=4)


In [ ]:
#tokenizers and models bert
bert_tokenizer= BertTokenizer.from_pretrained('bert-base-uncased')
bert_model= BertModel.from_pretrained('bert-base-uncased')


In [ ]:
#tokenizers and models t5
t5_tokenizer= T5Tokenizer.from_pretrained('t5-small')
t5_model= T5ForConditionalGeneration.from_pretrained('t5-small')

#move to device
device= 'cuda' if torch.cuda.is_available() else 'cpu'
bert_model.to(device)
t5_model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
# Training arguments
training_args= TrainingArguments(
  output_dir='/content/drive/My Drive/Implementation/Final Thesis/FYP/JSON generation/T5 CROSS NESTED',
  num_train_epochs=5,
  #training batch size
  per_device_train_batch_size=16,
  #evaluation batch size
  per_device_eval_batch_size=16,
  save_steps=500,
  save_total_limit=2,
  logging_steps=10,
  eval_steps=500,
  evaluation_strategy="steps",
  prediction_loss_only=True,
  load_best_model_at_end=True,
  fp16=torch.cuda.is_available(),
  #not push to HF
  push_to_hub=False
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
#datacollator for seq2seq
data_collator = DataCollatorForSeq2Seq(tokenizer=t5_tokenizer, model=t5_model)

In [ ]:
class T5WithCrossAttention(T5ForConditionalGeneration):
  """ class for T5 model with cross attention """
  def __init__(self, config):
    """ method to initialize the class
    Parameters:
    config (dict): configuration of the model
    """
    super().__init__(config)
    #initialise multi head attention layer for cross attention
    self.cross_attention = torch.nn.MultiheadAttention(embed_dim=config.d_model, num_heads=8)
    #initialise linear layer to put out bert embeddings to t5 hidden state dimension
    self.bert_proj = torch.nn.Linear(bert_model.config.hidden_size, config.d_model)

  def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, labels=None, bert_embeddings=None):
    """ method to forward the input
    Parameters:
    input_ids (tensor): input tokens
    attention_mask (tensor): attention mask
    decoder_input_ids (tensor): decoder input tokens
    labels (tensor): labels
    bert_embeddings (tensor): bert embeddings
    Returns:
    tuple: loss and logits
    """
    #pass inputs through t5 encoder
    outputs= self.encoder(input_ids=input_ids,attention_mask=attention_mask,)
    hidden_states= outputs[0]

    #bert embeddings to match the T5 hidden state dimension
    bert_embeddings = self.bert_proj(bert_embeddings).unsqueeze(1).expand(-1, hidden_states.size(1), -1)

    #putting cross-attention between T5 embeddings and BERT embeddings
    hidden_states, _ = self.cross_attention(hidden_states, bert_embeddings, bert_embeddings)

    #pass inputs through t5 decoder
    decoder_outputs= self.decoder(input_ids=decoder_input_ids,encoder_hidden_states=hidden_states,encoder_attention_mask=attention_mask,)

    sequence_output = decoder_outputs[0]
    lm_logits = self.lm_head(sequence_output)

    loss= None
    if labels is not None:
      loss_fct= torch.nn.CrossEntropyLoss(ignore_index=-100)
      #flatten logits and labels for loss computation
      lm_logits = lm_logits.view(-1, lm_logits.size(-1))
      #labels
      labels= labels.view(-1)
      if lm_logits.size(0)== labels.size(0):
        loss = loss_fct(lm_logits, labels)
      else:
        print(f"Shape mismatch between logits and labels: {lm_logits.size()} vs {labels.size()}")

    return loss, lm_logits


In [ ]:
#Trainer instance
trainer = Trainer(
  model=t5_model,
  args=training_args,
  train_dataset=train_dataset,
  eval_dataset=val_dataset,
  data_collator=data_collator
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss
500,0.804200,0.617622
1000,0.581100,0.468396
1500,0.501400,0.391192
2000,0.439100,0.341854
2500,0.402200,0.306227
3000,0.386600,0.278159
3500,0.363200,0.256101
4000,0.350200,0.240203
4500,0.320300,0.227243
5000,0.308500,0.216909


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=7060, training_loss=0.46850257328481915, metrics={'train_runtime': 6577.9211, 'train_samples_per_second': 17.173, 'train_steps_per_second': 1.073, 'total_flos': 1.528820989427712e+16, 'train_loss': 0.46850257328481915, 'epoch': 5.0})

In [ ]:
#path
model_save_path= '/content/drive/My Drive/Implementation/Final Thesis/FYP/JSON generation/T5 CROSS NESTED'

In [ ]:
#save
t5_model.save_pretrained(model_save_path)

In [ ]:
t5_tokenizer.save_pretrained(model_save_path)

('/content/drive/My Drive/Implementation/Final Thesis/FYP/JSON generation/T5 CROSS NESTED/tokenizer_config.json',
 '/content/drive/My Drive/Implementation/Final Thesis/FYP/JSON generation/T5 CROSS NESTED/special_tokens_map.json',
 '/content/drive/My Drive/Implementation/Final Thesis/FYP/JSON generation/T5 CROSS NESTED/spiece.model',
 '/content/drive/My Drive/Implementation/Final Thesis/FYP/JSON generation/T5 CROSS NESTED/added_tokens.json')

In [ ]:
def load_model_and_tokenizer(model_path):
  """ method to load the model and tokenizer
  Parameters:
  model_path (str): path to the model
  Returns:
  tuple: model and tokenizer
  """
  model = T5ForConditionalGeneration.from_pretrained(model_path)
  tokenizer = T5Tokenizer.from_pretrained(model_path)
  return model, tokenizer

In [ ]:
model, tokenizer = load_model_and_tokenizer(model_save_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def generate_json(prompt, model, tokenizer, max_length=1024):
  """method to generate json from prompt
  Parameters:
  prompt (str): prompt to generate json
  model (T5ForConditionalGeneration): model to generate json
  tokenizer (T5Tokenizer): tokenizer to generate json
  max_length (int): maximum length of the generated json
  Returns:
  str: generated json
  """
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model.to(device)
  #tokenize input description
  inputs= tokenizer(prompt, return_tensors='pt', padding='max_length', truncation=True, max_length=max_length).to(device)
   #generate JSON output
  outputs= model.generate(inputs['input_ids'], max_length=max_length,num_beams=5,early_stopping=True,temperature=0)
  #decode generated tokens to JSON string
  generated_json_str= tokenizer.decode(outputs[0], skip_special_tokens=True)
  return generated_json_str

In [ ]:
description="Create a trendy icon-button with a state of hover"

In [ ]:
generated_json= generate_json(description, model, tokenizer)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:
print(generated_json)

"variant_properties": "style": "Trendy", "component_name": "icon-button", "subtype": "Default", "variant_details": "State": ["Hover"], "name": "State=Hover", "type": "COMPONENT", "children": ["name": "Content", "type": "FRAME", "width": 32.0, "height": 32.0, "fills": [], "strokes": [], "effects": [], "borderRadius": 0, "strokeWeight": 1.0, "children": ["name": "Icon / SearchOutlined", "type": "INSTANCE", "children": ["name": "Vector", "type": "VECTOR", "iconWidth": 12.500054359436035, "iconHeight": 12.500054359436035, "iconColor": "r": 1.0, "g": 1.0, "b": 1.0, "a": 1.0], "name": "Button", "type": "TEXT", "fontFamily": "SF Pro Text", "fontWeight": 400, "fontSize": 14.0, "characters": "Button", "textColor": "r": 1.0, "g": 1.0, "b": 1.0, "a": 1.0]]
